In [1]:
import os
import time
import glob
import re
from functools import reduce
import numpy as np
import uproot
import uproot_methods
import awkward
import pandas as pd
from klepto.archives import dir_archive


import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray

%matplotlib inline
import matplotlib.pyplot as plt



In [2]:
def addRowToCutFlow( output, df, name, selection):
    '''
    add one row with name and selection for each process to the cutflow accumulator
    '''

    if selection is not None:
        output['cutflow'][name] += ( sum(df['weight'][selection].flatten() )*137)
    else:
        output['cutflow'][name] += ( sum(df['weight'].flatten() )*137)
            
def getCutFlowTable(output, outputname, lines = ['all']):
    '''
    Takes a cache and returns a formated cut-flow table of processes.
    Lines and processes have to follow the naming of the coffea processor output.
    '''
    res = pd.DataFrame([])
    for line in lines:
        res = res.append(pd.DataFrame({outputname : output['cutflow'][line]}, index=[line]))
    return res


In [3]:
#define processor

class WHhadProcessor(processor.ProcessorABC):
    def __init__(self):
        
        #define bins
        dataset_axis         = hist.Cat("dataset", "Primary dataset")
        pt_axis              = hist.Bin("pt", r"$p_{T}$ (GeV)", 500, 0, 2000)
        mass_axis            = hist.Bin("mass", r"$mass$ (GeV)", 500, 0, 2000)
        eta_axis             = hist.Bin("eta", r"$\eta$", 80, -4, 4)
        phi_axis             = hist.Bin("phi", r"$\Delta \phi$", 80, 0, 4)
        r_axis             = hist.Bin("phi", r"$\Delta R", 80, 0, 4)
        multiplicity_axis    = hist.Bin("multiplicity", r"N", 30, -0.5, 29.5)
        
        #define dictionary for plots
        self._accumulator = processor.dict_accumulator({
            'cutflow':              processor.defaultdict_accumulator(int),
            })

    
    @property
    def accumulator(self):
        return self._accumulator

    
    def process(self, df):
        
        output = self.accumulator.identity()
        
        dataset = df['dataset']

        #MET
        metphi = df["MET_phi"]
        metpt = df["MET_pt"]
        
        #leptons
        
        electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'].content, 
            eta=df['Electron_eta'].content, 
            phi=df['Electron_phi'].content,
            mass=df['Electron_mass'].content,
            pdgid=df['Electron_pdgId'].content,
            mini_iso=df['Electron_miniPFRelIso_all'].content
        )
        
        
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'].content, 
            eta=df['Muon_eta'].content, 
            phi=df['Muon_phi'].content,
            mass=df['Muon_mass'].content,
            pdgid=df['Muon_pdgId'].content,
            mini_iso=df['Muon_miniPFRelIso_all'].content, 
            looseid =df['Muon_looseId'].content
        )
        
        taus = JaggedCandidateArray.candidatesfromcounts(
            df['nTau'],
            pt=df['Tau_pt'].content, 
            eta=df['Tau_eta'].content, 
            phi=df['Tau_phi'].content,
            mass=df['Tau_mass'].content,
            decaymode=df['Tau_idDecayMode'].content,
            newid=df['Tau_idMVAnewDM2017v2'].content,
        )
        
        isotracks = awkward.JaggedArray.zip(
            pt=df['IsoTrack_pt'], 
            eta=df['IsoTrack_eta'], 
            phi=df['IsoTrack_phi'], 
            rel_iso=df['IsoTrack_pfRelIso03_all'], 
        )
        
        #jets
        jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'].content, 
            eta=df['Jet_eta'].content, 
            phi=df['Jet_phi'].content,
            btag=df['Jet_btagDeepB'].content, 
            jetid=df['Jet_jetId'].content, 
            mass=df['Jet_mass'].content,
        )
        fatjets = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt=df['FatJet_pt'].content, 
            eta=df['FatJet_eta'].content, 
            phi=df['FatJet_phi'].content, 
            mass=df['FatJet_mass'].content, 
            softdrop=df["FatJet_msoftdrop"].content,  
            fromH = df['FatJet_deepTagMD_HbbvsQCD'].content, 
            fromW_MD = df['FatJet_deepTagMD_WvsQCD'].content, 
            fromW_MC = df['FatJet_deepTag_WvsQCD'].content
        )
        
        #ak4's
        goodjcut = ((jets.pt>30) & (abs(jets.eta)<2.4) & (jets.jetid>0))
        goodjets = jets[goodjcut]
        njets = goodjets.counts
        jetpt_sorted = goodjets.pt.argsort(ascending=False)
        leadjet = goodjets[jetpt_sorted==0]
        subleadjet = goodjets[jetpt_sorted==1]
        leadjets = goodjets[jetpt_sorted <= 1]
      
        bjcut = ((jets.pt>30) & (abs(jets.eta)<2.4) & (jets.jetid>0) & (jets.btag>0.4184))
        bjets = jets[bjcut]
        numbjets = bjets.counts
        bjetpt = bjets.pt
        bjetpt_sorted = bjetpt.argsort(ascending=False)
        leadbjet = bjets[bjetpt_sorted==0]
        subleadbjet = bjets[bjetpt_sorted==1]
        leadbjets = bjets[bjetpt_sorted <= 1]
            
        nonbjcut = ((jets.pt>30) & (abs(jets.eta)<2.4) & (jets.jetid>0) & (jets.btag<0.4184))
        nbjets = jets[nonbjcut]
        nbjetpt_sorted = nbjets.pt.argsort(ascending=False)
        leadnbjet = nbjets[nbjetpt_sorted==0]
        subleadnbjet = nbjets[nbjetpt_sorted==1]
        leadnbjets = nbjets[nbjetpt_sorted <= 1]

        
        #ak8's
        goodfjcut = ((fatjets.pt > 200))
        goodfatjets = fatjets[goodfjcut]
        
        htagcut = ((fatjets.pt > 200) & (fatjets.fromH > 0.8365))
        htagged = fatjets[htagcut]
        htagpt_sorted = htagged.pt.argsort(ascending=False)
        leadhtag = htagged[htagpt_sorted==0]
        subleadhtag = htagged[htagpt_sorted==1]
        leadhtags = htagged[htagpt_sorted <= 1]
        
        wtagcut_mc = ((fatjets.pt > 200) & (fatjets.fromW_MC > 0.918) & (fatjets.fromH < 0.8365))
        wtagcut_md = ((fatjets.pt > 200) & (fatjets.fromW_MD > 0.704) & (fatjets.fromH < 0.8365))
        wtagged_mc = fatjets[wtagcut_mc]
        wtagged_md = fatjets[wtagcut_md]
        wtagmcpt_sorted = wtagged_mc.pt.argsort(ascending=False)
        leadwmctag = wtagged_mc[wtagmcpt_sorted==0]
        subleadwmctag = wtagged_mc[wtagmcpt_sorted==1]
        leadwmctags = wtagged_mc[wtagmcpt_sorted <= 1]
        wtagmdpt_sorted = wtagged_md.pt.argsort(ascending=False)
        leadwmdtag = wtagged_md[wtagmdpt_sorted==0]
        subleadwmdtag = wtagged_md[wtagmdpt_sorted==1]
        leadwmdtags = wtagged_md[wtagmdpt_sorted <= 1]
 
        WmcH = leadwmctag.cross(leadhtag)
        WmcHpt_sorted = WmcH.pt.argsort(ascending=False)
        leadWmcleadH = WmcH[WmcHpt_sorted <=1]

        WmdH = leadwmdtag.cross(leadhtag)
        WmdHpt_sorted = WmdH.pt.argsort(ascending=False)
        leadWmdleadH = WmdH[WmdHpt_sorted <=1]

        #match jets
        
        leadjleadH = leadhtag.cross(leadjet)
        leadbjleadH = leadhtag.cross(leadbjet)
        leadnbjleadH = leadhtag.cross(leadnbjet)
        
        subleadjleadH = leadhtag.cross(subleadjet)
        subleadbjleadH = leadhtag.cross(subleadbjet)
        subleadnbjleadH = leadhtag.cross(subleadnbjet)
        
        leadjleadW = leadwmctag.cross(leadjet)
        leadbjleadW = leadwmctag.cross(leadbjet)
        leadnbjleadW = leadwmctag.cross(leadnbjet)
        
        subleadjleadW = leadwmctag.cross(leadjet)
        subleadbjleadW = leadwmctag.cross(leadbjet)
        subleadnbjleadW = leadwmctag.cross(leadnbjet)
        
        #angles
        
        leadjleadHDR = leadjleadH.i0.p4.delta_r(leadjleadH.i1.p4)
        leadjleadHDPhi = abs(leadjleadH.i0.p4.delta_phi(leadjleadH.i1.p4))
        leadbjleadHDR = leadbjleadH.i0.p4.delta_r(leadbjleadH.i1.p4)
        leadbjleadHDPhi = abs(leadbjleadH.i0.p4.delta_phi(leadbjleadH.i1.p4))
        leadnbjleadHDR = leadnbjleadH.i0.p4.delta_r(leadnbjleadH.i1.p4)
        leadnbjleadHDPhi = abs(leadnbjleadH.i0.p4.delta_phi(leadnbjleadH.i1.p4))
        
        subleadjleadHDR = subleadjleadH.i0.p4.delta_r(subleadjleadH.i1.p4)
        subleadjleadHDPhi = abs(subleadjleadH.i0.p4.delta_phi(subleadjleadH.i1.p4))
        subleadbjleadHDR = subleadbjleadH.i0.p4.delta_r(subleadbjleadH.i1.p4)
        subleadbjleadHDPhi = abs(subleadbjleadH.i0.p4.delta_phi(subleadbjleadH.i1.p4))
        subleadnbjleadHDR = subleadnbjleadH.i0.p4.delta_r(subleadnbjleadH.i1.p4)
        subleadnbjleadHDPhi = abs(subleadnbjleadH.i0.p4.delta_phi(subleadnbjleadH.i1.p4))
        
        leadjleadWDR = leadjleadW.i0.p4.delta_r(leadjleadW.i1.p4)
        leadjleadWDPhi = abs(leadjleadW.i0.p4.delta_phi(leadjleadW.i1.p4))
        leadbjleadWDR = leadbjleadW.i0.p4.delta_r(leadbjleadW.i1.p4)
        leadbjleadWDPhi = abs(leadbjleadW.i0.p4.delta_phi(leadbjleadW.i1.p4))
        leadnbjleadWDR = leadnbjleadW.i0.p4.delta_r(leadnbjleadW.i1.p4)
        leadnbjleadWDPhi = abs(leadnbjleadW.i0.p4.delta_phi(leadnbjleadW.i1.p4))
        
        subleadjleadWDR = subleadjleadW.i0.p4.delta_r(subleadjleadW.i1.p4)
        subleadjleadWDPhi = abs(subleadjleadW.i0.p4.delta_phi(subleadjleadW.i1.p4))
        subleadbjleadWDR = subleadbjleadW.i0.p4.delta_r(subleadbjleadW.i1.p4)
        subleadbjleadWDPhi = abs(subleadbjleadW.i0.p4.delta_phi(subleadbjleadW.i1.p4))
        subleadnbjleadWDR = subleadnbjleadW.i0.p4.delta_r(subleadnbjleadW.i1.p4)
        subleadnbjleadWDPhi = abs(subleadnbjleadW.i0.p4.delta_phi(subleadnbjleadW.i1.p4))
        
        
        #match phi
        dphi_leadj_met = abs((leadjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        dphi_leadbj_met = abs((leadbjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        dphi_leadnbj_met = abs((leadnbjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        
        dphi_subleadj_met = abs((subleadjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        dphi_subleadbj_met = abs((subleadbjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        dphi_subleadnbj_met = abs((subleadnbjet.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        
        dphi_h_met = abs((leadhtag.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        dphi_w_met = abs((leadwmctag.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        
        
        #MT      
        
        dphi_leadbs_met = abs((leadbjets.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        mt_b_met = np.sqrt(2*leadbjets.pt*metpt*(1-np.cos(dphi_leadbs_met)))

        
        sorted_min_mt_b_met = mt_b_met.argsort(ascending=True)
        sorted_max_mt_b_met = mt_b_met.argsort(ascending=False)
        min_mt_b_met = mt_b_met[sorted_min_mt_b_met == 0]
        max_mt_b_met = mt_b_met[sorted_max_mt_b_met == 0]

        
        #leptons
        
        veto_e_cut = (electrons.pt>5) & (abs(electrons.eta) < 2.4) & (electrons.mini_iso < 0.2)
        veto_e = electrons[veto_e_cut]
        
        veto_m_cut = (muons.pt > 5) & (abs(muons.eta) < 2.4) & (muons.looseid) & (muons.mini_iso < 0.2)
        veto_m = muons[veto_m_cut]
        
        veto_t_cut = (taus.pt > 20) & (abs(taus.eta) < 2.4) & (taus.decaymode) & (taus.newid >= 8)
        veto_t = taus[veto_t_cut]
        
        veto_it_cut = (isotracks.pt > 10) & (abs(isotracks.eta) < 2.4) & ((isotracks.rel_iso < (0.1*isotracks.pt)) | (isotracks.rel_iso < 6))
        veto_it = isotracks[veto_it_cut]
        
    
        #DPHI
        dphi_met_leadjs = abs((leadjets.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        sorted_dphi_met_leadjs = dphi_met_leadjs.argsort(ascending=True)
        min_dphi_met_leadjs = dphi_met_leadjs[sorted_dphi_met_leadjs==0]
        abs_min_dphi_met_leadjs = abs(min_dphi_met_leadjs)
        
        dphi_met_wmch = abs((leadWmcleadH.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        sorted_dphi_met_wmch = dphi_met_wmch.argsort(ascending=True)
        min_dphi_met_wmch = dphi_met_wmch[sorted_dphi_met_wmch==0]
        abs_min_dphi_met_wmch = abs(min_dphi_met_wmch)
        
        dphi_met_wmdh = abs((leadWmdleadH.phi - metphi + np.pi) % (2 * np.pi) - np.pi)
        sorted_dphi_met_wmdh = dphi_met_wmdh.argsort(ascending=True)
        min_dphi_met_wmdh = dphi_met_wmdh[sorted_dphi_met_wmdh==0]
        abs_min_dphi_met_wmdh = abs(min_dphi_met_wmdh)
        

        #PRESELECTION
        ht = goodjets.pt.sum()
        
        met_ps = (df['MET_pt']>400)
        njet_ps = (goodjets.counts >= 2)
        bjet_ps = (bjets.counts >= 1)
        fatjet1_ps = (goodfatjets.counts >= 1)
        fatjet2_ps = (goodfatjets.counts >= 2)
        ht_ps = (ht > 300)
        
        presel = met_ps & njet_ps & bjet_ps  & ht_ps
        h_sel =(htagged.counts > 0) 
        wmd_sel = (wtagged_md.counts > 0) 
        wmc_sel = (wtagged_mc.counts > 0) 
        e_sel = (veto_e.counts == 0)
        m_sel = (veto_m.counts == 0)
        it_sel = (veto_it.counts == 0)
        t_sel = (veto_t.counts == 0)
        mt_sel = (min_mt_b_met > 200).any()

        
        #cumulative preselection
        
        sel1 =  e_sel
        sel2 =  e_sel & m_sel
        sel3 = e_sel & m_sel & t_sel
        sel4 = e_sel & m_sel & t_sel & it_sel
        sel5 = e_sel & m_sel & t_sel & it_sel & met_ps
        sel6 = e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps
        sel7 = e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps 
        sel8 =  e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps
        sel9 =  e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & mt_sel
        sel13 =  e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & mt_sel & fatjet1_ps
        sel14 =  e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & mt_sel & fatjet2_ps
        sel10 =  e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & h_sel & mt_sel & fatjet2_ps
        sel11 = e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & h_sel & wmd_sel & mt_sel & fatjet2_ps
        sel12 = e_sel & m_sel & t_sel & it_sel & met_ps & ht_ps & njet_ps & bjet_ps & h_sel & wmc_sel & mt_sel & fatjet2_ps

        #DEFINE VARIABLES 
    
    
        wght = df['weight'][0] * 137 *1000

        #FILL OUTPUTS

        #raw cutflow
                
        addRowToCutFlow(output, df, 'all', None)
        addRowToCutFlow(output, df, '+ e veto', sel1)
        addRowToCutFlow(output, df, '+ m veto', sel2)
        addRowToCutFlow(output, df, '+ t veto', sel3)
        addRowToCutFlow(output, df, '+ it veto', sel4)
        addRowToCutFlow(output, df, '+ met', sel5)
        addRowToCutFlow(output, df, '+ ht', sel6)
        addRowToCutFlow(output, df, '+ njet', sel7)
        addRowToCutFlow(output, df, '+ bjet', sel8)
        addRowToCutFlow(output, df, '+ min(mt(bs,met))', sel9)
        addRowToCutFlow(output, df, '+ fatjet >=1 ', sel13)
        addRowToCutFlow(output, df, '+ fatjet >=2 ', sel14)
        addRowToCutFlow(output, df, '+ h > 0', sel10)
        addRowToCutFlow(output, df, '+ w(MC) > 0', sel12)
        
     
        return output

    
    def postprocess(self, accumulator):
        return accumulator
        

In [4]:
fileset_sig = {'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/WH_had_750_1_nanoAOD/*.root')}
fileset_wjets = {'WJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/WJetsToLNu*/*.root')}
fileset_qcd = {'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/QCD_HT*/*.root')}
fileset_ttjets = {'TTJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/TTJets*/*.root')}
fileset_znunu = {'ZNuNu': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ZJetsToNuNu*/*.root')}
fileset_st = {'ST':glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ST*/*.root')}
fileset_ttwz = {'ttW/ttZ':glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ttWJets*/*.root')
                +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ttZJets*/*.root')}
fileset_2b = {'2bosons':glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/WW*/*.root')
                +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/WZ*/*.root')
                +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ZZTo2L2Nu*/*.root')
                +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/0p1p16/2018/ZZTo2Q2Nu*/*.root')}

output_sig = processor.run_uproot_job(fileset_sig,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_wjets = processor.run_uproot_job(fileset_wjets,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 ) 
output_qcd = processor.run_uproot_job(fileset_qcd,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_ttjets = processor.run_uproot_job(fileset_ttjets,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_znunu = processor.run_uproot_job(fileset_znunu,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_st = processor.run_uproot_job(fileset_st,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_ttwz = processor.run_uproot_job(fileset_ttwz,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )
output_2b = processor.run_uproot_job(fileset_2b,
                                    treename='Events',
                                    processor_instance=WHhadProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 24, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )

In [5]:
getCutFlowTable(output_sig, 'mC750_l1', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet',  '+ min(mt(bs,met))',
                                                '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])



,mC750_l1
all,297.956555
+ e veto,269.654399
+ m veto,263.477663
+ t veto,259.956788
+ it veto,224.734525
+ met,138.908981
+ ht,138.591359
+ njet,138.341316
+ bjet,115.222483
"+ min(mt(bs,met))",106.389885


In [6]:
getCutFlowTable(output_wjets, 'W+Jets', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet',  '+ min(mt(bs,met))',
                                                 '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,W+Jets
all,6.478594e+06
+ e veto,4.364317e+06
+ m veto,2.608754e+06
+ t veto,2.114283e+06
+ it veto,1.698079e+06
+ met,6.797472e+04
+ ht,6.709634e+04
+ njet,6.507965e+04
+ bjet,6.919150e+03
"+ min(mt(bs,met))",4.126043e+03


In [7]:
getCutFlowTable(output_qcd, 'QCD', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet',  '+ min(mt(bs,met))',
                                             '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,QCD
all,3.910187e+06
+ e veto,3.545849e+06
+ m veto,2.826098e+06
+ t veto,2.796073e+06
+ it veto,2.426299e+06
+ met,5.580653e+04
+ ht,5.408350e+04
+ njet,5.406641e+04
+ bjet,9.999168e+03
"+ min(mt(bs,met))",4.576661e+03


In [8]:
getCutFlowTable(output_ttjets, 'TTJets', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet', '+ min(mt(bs,met))',
                                                  '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0','+ w(MC) > 0'])

,TTJets
all,928788.070357
+ e veto,555277.764814
+ m veto,292381.434678
+ t veto,236252.153177
+ it veto,163987.196773
+ met,5680.001163
+ ht,5657.329252
+ njet,5650.375109
+ bjet,4684.874819
"+ min(mt(bs,met))",1495.204793


In [9]:
getCutFlowTable(output_znunu, 'ZNuNu', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet', '+ min(mt(bs,met))',
                                                '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,ZNuNu
all,1.749745e+06
+ e veto,1.616261e+06
+ m veto,1.595967e+06
+ t veto,1.579711e+06
+ it veto,1.422254e+06
+ met,1.054298e+05
+ ht,1.042135e+05
+ njet,9.926064e+04
+ bjet,1.004382e+04
"+ min(mt(bs,met))",8.408002e+03


In [10]:
getCutFlowTable(output_st, 'ST', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet', '+ min(mt(bs,met))',
                                                '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,ST
all,291963.198720
+ e veto,176664.751700
+ m veto,93536.186919
+ t veto,71731.301404
+ it veto,51533.189482
+ met,1890.616657
+ ht,1869.303024
+ njet,1859.732957
+ bjet,1413.214014
"+ min(mt(bs,met))",661.690065


In [11]:
getCutFlowTable(output_ttwz, 'ttw/ttz', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet', '+ min(mt(bs,met))',
                                                '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,ttw/ttz
all,11906.833334
+ e veto,7193.408606
+ m veto,4278.290829
+ t veto,3686.124259
+ it veto,2565.326204
+ met,259.734882
+ ht,258.881358
+ njet,258.791247
+ bjet,220.221061
"+ min(mt(bs,met))",149.708159


In [12]:
getCutFlowTable(output_2b, 'WW/WZ/ZZ', lines = ['all', '+ e veto', '+ m veto', '+ t veto', 
                                                '+ it veto', '+ met', '+ ht',
                                                '+ njet', '+ bjet', '+ min(mt(bs,met))',
                                                '+ fatjet >=1 ', '+ fatjet >=2 ',
                                                '+ h > 0', '+ w(MC) > 0'])

,WW/WZ/ZZ
all,90495.015945
+ e veto,63289.550567
+ m veto,43366.535236
+ t veto,38078.108888
+ it veto,31387.806067
+ met,2321.230589
+ ht,2285.407665
+ njet,2182.469835
+ bjet,336.158848
"+ min(mt(bs,met))",268.003356
